# TP 2: Agent Réactif dans une Grille

## Hany El Atlassi CSCC TP 2


## **Objectifs du TP**
- Comprendre la prise de décision locale dans un environnement inconnu.
- Implémenter un agent réactif se déplaçant dans une grille.
- Simuler les capteurs (murs, obstacles) et les actions (avancer, tourner).
- Observer les limites des agents sans mémoire.

Un robot est placé dans une grille (grid) de taille 8×8. Chaque case vaut 0 (libre) ou 1 (mur / wall)  
Le robot commence en S = (1, 1) et doit atteindre G = (6, 6)  
Il peut se déplacer Nord, Est, Sud, Ouest  
Il suit une priorité : droite → avant → gauche → demi-tour  

Exemple de labyrinth:
<b><center>LAB = [ </center></b>
<b><center>[1,1,1,1,1,1,1,1],</center></b>
<b><center>[1,0,0,0,1,0,0,1],</center></b>
<b><center>[1,0,1,0,0,0,1,1],</center></b>
<b><center>[1,0,1,0,1,0,0,1],</center></b>
<b><center>[1,0,0,0,1,1,0,1],</center></b>
<b><center>[1,1,1,0,0,0,0,1],</center></b>
<b><center>[1,0,0,0,1,0,0,1],</center></b>
<b><center>[1,1,1,1,1,1,1,1]</center></b>
<b><center>]</center></b>

## Partie 1: Capteurs et Mouvements

Dans cette partie, vous allez implémenter les ofnctions de base permettant au robot de percevoir son environnement et de se déplacer.

**Q 1.1**: Implémentation des fonctions de base

In [1]:
DIRS = [(-1,0),(0,1),(1,0),(0,-1)] # N,E,S,O

def in_bounds(G,i,j):
    return True if (0 <= i < len(G)) and (0 <= j < len(G[0])) else False

def is_wall(G,i,j):
    return True if G[i][j] == 1 or in_bounds(G,i,j) == False else False

def move(i,j,d):
    return (i + DIRS[d][0], j + DIRS[d][1])

# Il faut vérifier in_bounds et is_wall avant de faire ce mouvement

**Q 1.2**: Quelle est la différence entre in_bounds et is_wall?

La fonction in_bounds vérifie si les coordonées (i,j) sont correct dans la grille.
La fonction is_wall nous aide à détecter si la position (i,j) est un mur dans le labyrinthe ou non.
On a besoin des deux pour définir le mouvement de l'agent dans le labyrinthe.

**Q 1.3**: Comportement hors limites
Si le robot essaie de sortir de la grille, l'action de "SE DEPLACER" est tout simplement bloquée avant etre achevée.

## **Partie 2: Décision Locale (Règles Simples)**
Dans cette partie, vous allez implémenter la stratégie de décision de l'agent réactif basée sur la règle de la "main droite".

**Q 2.1**: Implémentation de choose_action

In [2]:
def choose_action(G,i,j,d):
    ACTIONS = ["DROITE","AVANT","GAUCHE","DEMI-TOUR"]
    for action in ACTIONS:
        if action == "DROITE":
            new_d = DIRS[(d+1)%4]
            new_i,new_j = i + new_d[0], j + new_d[1]
            if not is_wall(G,new_i,new_j):
                return (action, (d+1)%4, new_i, new_j)
        if action == "AVANT":
            new_d = DIRS[d]
            new_i, new_j = i + new_d[0], j + new_d[1]
            if not is_wall(G,new_i,new_j):
                return (action, d, new_i, new_j)
        if action == "GAUCHE":
            new_d = DIRS[(d-1)%4]
            new_i, new_j = i + new_d[0], j + new_d[1]
            if not is_wall(G,new_i,new_j):
                return (action, (d-1)%4, new_i, new_j)
        if action == "DEMI-TOUR":
            new_d = DIRS[(d+2)%4]
            new_i, new_j = i + new_d[0], j + new_d[1]
            if not is_wall(G,new_i,new_j):
                return (action, (d+2)%4, new_i, new_j)
    return None
    


**Q 2.2**: Description de la stratégie

Le choix de l'action suit l'ordre DROITE AVANT GAUCHE DEMI-TOUR, donc on donne à chaque tour la priorrité au mouvement à droite. On teste tout d'abords si ce choix est possible en vérifiant si la nouvelle position mène dehors le labyrinthe ou mène vers un mur. Si ce n'est pas le cas, cette action est alors choisie et retournée.

**Q 2.3**: Ordre de priorité

Ceci simule l'algorithme de recherche de graphe qui suit un mur jusqu'à impossibilité de le suivre, et on change le mur suivi vers le suivant dans l'ordre. Cela permet de traverser le labyrinthe dans sa totalité (ou si on est chanceux, on trouve la sortie avant cela).

**Q 2.4**: La seule situation ou c'est un cas bloqué est si la position initiale de l'agent est dans une case vide encerclée par des murs. La fonction retourne None. Sinon ce cas n'est pas possible on est toujours capable de faire demi-tour.

**Q 2.5**: Variante "main gauche"

La seule modification serait de changer l'ordre de DIRS pour aller comme ci: GAUCHE AVANT DROITE DEMI-TOUR

## **Partie 3: Simulation du Déplacement**

**Q 3.1**: Implémentation de run_agent

In [3]:
def run_agent(G,start,goal,d0=0,max_steps=200):
    i,j = start
    d = d0
    path = []
    for _ in range(max_steps):
        if choose_action(G,i,j,d) != None:
            action, d, i, j = choose_action(G,i,j,d)
            path.append(action)
            if (i,j) == goal:
                return path
        else:
            return "L'Agent est bloqué"
    

In [4]:
G = (6,6)
S = (1,1)
LAB = [
[1,1,1,1,1,1,1,1],
[1,0,0,0,1,0,0,1],
[1,0,1,0,0,0,1,1],
[1,0,1,0,1,0,0,1],
[1,0,0,0,1,1,0,1],
[1,1,1,0,0,0,0,1],
[1,0,0,0,1,0,0,1],
[1,1,1,1,1,1,1,1]
]

path = run_agent(LAB,S,G,d0=1)
print("Length of path: ", len(path))

Length of path:  16


**Q 3.2**: Résultats de l'éxecution

Le nombre d'étapes pour atteindre le but: 16  
La longeur du chemin parcouru: 16  
Le comportement observé:

In [5]:
print(path)

['DROITE', 'AVANT', 'AVANT', 'GAUCHE', 'AVANT', 'DROITE', 'AVANT', 'DROITE', 'AVANT', 'DEMI-TOUR', 'AVANT', 'GAUCHE', 'DROITE', 'AVANT', 'DROITE', 'GAUCHE']


**Q 3.3**: Le but est effectivement atteint. Le chemin n'est pas forcément optimal, ceci est du à l'algorithme utilisé, il est possible de passer par le milieu du labyrinthe, mais l'algorithme préfère la droite, ce qui veut dire les bords.

## **Partie 4: Visualitation du Chemin**

**Q 4**: Implémentation de print_path

In [6]:
def print_path(G,path,S,Ggoal):
    G_copy = [row[:] for row in G]
    i, j = S
    d = 1
    G_copy[i][j] = 'S'
    G_copy[Ggoal[0]][Ggoal[1]] = 'G'

    for action in path:
        if action == "DROITE":
            d = (d + 1) % 4
        elif action == "GAUCHE":
            d = (d - 1) % 4
        elif action == "DEMI-TOUR":
            d = (d + 2) % 4
        
        i, j = move(i, j, d)
        if (i, j) != Ggoal:
            G_copy[i][j] = '█'  
    for row in G_copy:
        print(' '.join(str(cell) for cell in row))

In [7]:
print_path(LAB, path, S, G)

1 1 1 1 1 1 1 1
1 S 0 0 1 0 0 1
1 █ 1 0 0 0 1 1
1 █ 1 0 1 0 0 1
1 █ █ █ 1 1 0 1
1 1 1 █ █ █ 0 1
1 █ █ █ 1 █ G 1
1 1 1 1 1 1 1 1


## Partie 5: Analyse et Réflexion

**Q 5.1**: La seule situation où l'agent n'atteinds pas le but est quand le but est loin d'un mur, cet algorithme dépend énormement de l'étroitesse du labyrinthe. Si ce dernier n'est pas étroit, l'agent peut tout simplement tourner en rond et ne jamais trouver l'objectif.

**Q 5.2**: Les boucles infinies
Les boucles infinies sont un phénomène qui se passe quand le labyrinthe n'est pas étroit. Dans notre programe c'est si l'agent arrive à "200 steps". On peut tout autant détecter si 4 action de meme nature s'éxecutent pour mieux détecter cette situation.

**Q 5.3**: L'agent peut utiliser une mémoire, ceci serait efficace pour ne pas redécouvrir une chemin déja découvert. On mémorisera alors les coordonnées passées, et essayer de modéliser un chemin bloqué. si ce chemin bloqué est déja découvert, pas la peine de le redécouvrir.

**Q 5.4**: L'agent peut suivre son algorithme de main droite, et quand il trouve que 2 ou 3 coordonnées successives (on met le point sur le fait qu'ils sont successifs, du meme ordre traversé ou en inverse) il arrete, et essaie un autre chemin, ou en faisant demi tour si aucun autre chemin existe.